In [69]:
ls

FDAH/               fastQC/             mapped/             sample_sheet.csv
annovar/            fastq/              pileup/             sample_sheet1.csv
config.yaml         fastq2/             realigned/          sheet1/
coverBED/           filtered/           recalib/            varscan/
deduped/            logs/               sampleFinder.ipynb


### filter for files with fastq extension

In [85]:
fastq_list = []
for folder, subfolders, files in os.walk('FDAH'):
    for file in files:
        if '.fastq.' in file and not '.md5' in file and not 'trim' in file and not 'sub' in file:
            fastq_list.append(os.path.join(folder,file))
fastq_list

['FDAH/FD_AH_001_002B/HL3KHDMXX/L001/FD_AH_001_002B_L001_R2_001.fastq.gz',
 'FDAH/FD_AH_001_002B/HL3KHDMXX/L001/FD_AH_001_002B_L001_R1_001.fastq.gz',
 'FDAH/FD_AH_001_002B/HL3KHDMXX/L001/FD_AH_001_002B_L001_barcode_1_001.fastq.gz',
 'FDAH/FD_AH_001_002B/HL3KHDMXX/L001/FD_AH_001_002B_L001_index_1_001.fastq.gz',
 'FDAH/FD_AH_002_013B/HL3KHDMXX/L002/FD_AH_002_013B_L002_barcode_1_001.fastq.gz',
 'FDAH/FD_AH_002_013B/HL3KHDMXX/L002/FD_AH_002_013B_L002_R2_001.fastq.gz',
 'FDAH/FD_AH_002_013B/HL3KHDMXX/L002/FD_AH_002_013B_L002_R1_001.fastq.gz',
 'FDAH/FD_AH_002_013B/HL3KHDMXX/L002/FD_AH_002_013B_L002_index_1_001.fastq.gz',
 'FDAH/FD_AH_002_008A/HL3KHDMXX/L002/FD_AH_002_008A_L002_barcode_1_001.fastq.gz',
 'FDAH/FD_AH_002_008A/HL3KHDMXX/L002/FD_AH_002_008A_L002_index_1_001.fastq.gz',
 'FDAH/FD_AH_002_008A/HL3KHDMXX/L002/FD_AH_002_008A_L002_R2_001.fastq.gz',
 'FDAH/FD_AH_002_008A/HL3KHDMXX/L002/FD_AH_002_008A_L002_R1_001.fastq.gz',
 'FDAH/FD_AH_002_014B/HL3KHDMXX/L002/FD_AH_002_014B_L002_R1_001.

### read in the sample sheet

In [86]:
samples = pd.read_csv('sample_sheet1.csv', sep='\t', index_col=0)
samples[:5]

,sample,Lane,name
16,FD_AH_002_008A,2,008_A
17,FD_AH_002_008B,2,008_B
18,FD_AH_002_009A,2,009_A
19,FD_AH_002_009B,2,009_B
20,FD_AH_002_011A,2,011_A


### function to retrieve fastqs for specific row of sample sheet

In [87]:
for file in fastq_list:
    print(file)

FDAH/FD_AH_001_002B/HL3KHDMXX/L001/FD_AH_001_002B_L001_R2_001.fastq.gz
FDAH/FD_AH_001_002B/HL3KHDMXX/L001/FD_AH_001_002B_L001_R1_001.fastq.gz
FDAH/FD_AH_001_002B/HL3KHDMXX/L001/FD_AH_001_002B_L001_barcode_1_001.fastq.gz
FDAH/FD_AH_001_002B/HL3KHDMXX/L001/FD_AH_001_002B_L001_index_1_001.fastq.gz
FDAH/FD_AH_002_013B/HL3KHDMXX/L002/FD_AH_002_013B_L002_barcode_1_001.fastq.gz
FDAH/FD_AH_002_013B/HL3KHDMXX/L002/FD_AH_002_013B_L002_R2_001.fastq.gz
FDAH/FD_AH_002_013B/HL3KHDMXX/L002/FD_AH_002_013B_L002_R1_001.fastq.gz
FDAH/FD_AH_002_013B/HL3KHDMXX/L002/FD_AH_002_013B_L002_index_1_001.fastq.gz
FDAH/FD_AH_002_008A/HL3KHDMXX/L002/FD_AH_002_008A_L002_barcode_1_001.fastq.gz
FDAH/FD_AH_002_008A/HL3KHDMXX/L002/FD_AH_002_008A_L002_index_1_001.fastq.gz
FDAH/FD_AH_002_008A/HL3KHDMXX/L002/FD_AH_002_008A_L002_R2_001.fastq.gz
FDAH/FD_AH_002_008A/HL3KHDMXX/L002/FD_AH_002_008A_L002_R1_001.fastq.gz
FDAH/FD_AH_002_014B/HL3KHDMXX/L002/FD_AH_002_014B_L002_R1_001.fastq.gz
FDAH/FD_AH_002_014B/HL3KHDMXX/L002/FD_AH_

In [88]:
def get_fastq_paths(row):
    for file in fastq_list:
        if row['sample'] in file:
            if 'index' in file:
                row['index'] = file
            elif 'barcode' in file:
                row['barcode'] = file
            elif '_R1' in file:
                row['R1'] = file
            elif '_R2' in file:
                row['R2'] = file
    return row[['name', 'Lane', 'R1', 'R2', 'barcode', 'index']]

### populate sample sheet with fastq paths

In [89]:
samples = samples.apply(get_fastq_paths, axis=1)
samples[:5]

,name,Lane,R1,R2,barcode,index
16,008_A,2,FDAH/FD_AH_002_008A/HL3KHDMXX/L002/FD_AH_002_008A_L002_R1_001.fastq.gz,FDAH/FD_AH_002_008A/HL3KHDMXX/L002/FD_AH_002_008A_L002_R2_001.fastq.gz,FDAH/FD_AH_002_008A/HL3KHDMXX/L002/FD_AH_002_008A_L002_barcode_1_001.fastq.gz,FDAH/FD_AH_002_008A/HL3KHDMXX/L002/FD_AH_002_008A_L002_index_1_001.fastq.gz
17,008_B,2,FDAH/FD_AH_002_008B/HL3KHDMXX/L002/FD_AH_002_008B_L002_R1_001.fastq.gz,FDAH/FD_AH_002_008B/HL3KHDMXX/L002/FD_AH_002_008B_L002_R2_001.fastq.gz,FDAH/FD_AH_002_008B/HL3KHDMXX/L002/FD_AH_002_008B_L002_barcode_1_001.fastq.gz,FDAH/FD_AH_002_008B/HL3KHDMXX/L002/FD_AH_002_008B_L002_index_1_001.fastq.gz
18,009_A,2,FDAH/FD_AH_002_009A/HL3KHDMXX/L002/FD_AH_002_009A_L002_R1_001.fastq.gz,FDAH/FD_AH_002_009A/HL3KHDMXX/L002/FD_AH_002_009A_L002_R2_001.fastq.gz,FDAH/FD_AH_002_009A/HL3KHDMXX/L002/FD_AH_002_009A_L002_barcode_1_001.fastq.gz,FDAH/FD_AH_002_009A/HL3KHDMXX/L002/FD_AH_002_009A_L002_index_1_001.fastq.gz
19,009_B,2,FDAH/FD_AH_002_009B/HL3KHDMXX/L002/FD_AH_002_009B_L002_R1_001.fastq.gz,FDAH/FD_AH_002_009B/HL3KHDMXX/L002/FD_AH_002_009B_L002_R2_001.fastq.gz,FDAH/FD_AH_002_009B/HL3KHDMXX/L002/FD_AH_002_009B_L002_barcode_1_001.fastq.gz,FDAH/FD_AH_002_009B/HL3KHDMXX/L002/FD_AH_002_009B_L002_index_1_001.fastq.gz
20,011_A,2,FDAH/FD_AH_002_011A/HL3KHDMXX/L002/FD_AH_002_011A_L002_R1_001.fastq.gz,FDAH/FD_AH_002_011A/HL3KHDMXX/L002/FD_AH_002_011A_L002_R2_001.fastq.gz,FDAH/FD_AH_002_011A/HL3KHDMXX/L002/FD_AH_002_011A_L002_barcode_1_001.fastq.gz,FDAH/FD_AH_002_011A/HL3KHDMXX/L002/FD_AH_002_011A_L002_index_1_001.fastq.gz


## get tumor-normal-pairs

In [95]:
def get_tumor_normal_pairs(samples):
    '''
    turns valid_file_info into tuples of valid tumor normal pairs
    '''

    # extract the tumor-normal suffix
    samples[['sample', 'TN']] = samples['name'].str.extract('(?P<sample>^[^_]+)_(?P<TN>[^_]+)$')
    tumor = 'A'
    normal = 'B'

    def TN_pair(group):
        print(group['name'])
        if tumor in list(group['TN']) and normal in list(group['TN']):
            return f"{group['sample'][0]}_{tumor}-{normal}"
        
    TN_df = pd.DataFrame()
    TN_df['pairs'] = samples.groupby('sample').apply(TN_pair)
    return list(TN_df['pairs'])

In [102]:
def TN_pair(group):
    print(group['name'])
    if tumor in list(group['TN']) and normal in list(group['TN']):
        return f"{group['sample'][0]}_{tumor}-{normal}"
    
samples.groupby('sample').apply(lambda df: df['name'])

sample    
008     16    008_A
        17    008_B
009     18    009_A
        19    009_B
011     20    011_A
        21    011_B
012     22    012_A
        23    012_B
013     24    013_A
        25    013_B
014     26    014_A
        27    014_B
015     28    015_A
        29    015_B
016     30    016_A
        31    016_B
Name: name, dtype: object

### get sample-type list for multiqc

In [40]:
def get_fastqc_list():
    fastqc_list = []
    trim_suffix = 'TRIM'
    # create file list from the included_files tuple list
    read1 = 'R1'
    read2 = 'R2'
    trim_suffix = 'TRIM'
    for file_name in samples['name']:
        fastqc_list.append(f"fastQC/{file_name}_{read1}_fastqc.zip")
        fastqc_list.append(f"fastQC/{file_name}_{read2}_fastqc.zip")
        if True: # config['trim']['trimming']:
            fastqc_list.append(f"fastQC/{file_name}_{read1}.{trim_suffix}_fastqc.zip")
            fastqc_list.append(f"fastQC/{file_name}_{read2}.{trim_suffix}_fastqc.zip")
    return fastqc_list

In [41]:
get_fastqc_list()

['fastQC/test_A_R1_fastqc.zip',
 'fastQC/test_A_R2_fastqc.zip',
 'fastQC/test_A_R1.TRIM_fastqc.zip',
 'fastQC/test_A_R2.TRIM_fastqc.zip',
 'fastQC/test_B_R1_fastqc.zip',
 'fastQC/test_B_R2_fastqc.zip',
 'fastQC/test_B_R1.TRIM_fastqc.zip',
 'fastQC/test_B_R2.TRIM_fastqc.zip']